In [5]:
import pandas as pd
import psycopg2

# Dictionnaire définissant le type de données pour chaque colonne
column_types = {
    "PassengerId": "INTEGER",
    "Survived": "Boolean",
    "Pclass": "INTEGER",
    "Name": "TEXT",
    "Sex": "TEXT",
    "Age": "REAL",
    "SibSp": "INTEGER",
    "Parch": "INTEGER",
    "Ticket": "TEXT",
    "Fare": "REAL",
    "Cabin": "TEXT",
    "Embarked": "TEXT"
}

# Récupération des noms de colonnes à partir des clés du dictionnaire
column_headers = column_types.keys()

# Générer la commande SQL avec les types appropriés
sql_command = "CREATE TABLE titanic (" + ", ".join([f"{col} {column_types[col]}" for col in column_headers]) + ");"

# Connexion à PostgreSQL et exécution de la commande
conn = psycopg2.connect("dbname='workshopdb' user='dev' password='1234'")
cur = conn.cursor()
cur.execute(sql_command)
conn.commit()
cur.close()
conn.close()


In [7]:
import csv
import psycopg2

# Connexion à la base de données
conn = psycopg2.connect("dbname='workshopdb' user='dev' password='1234'")
cur = conn.cursor()

# Calculer la moyenne d'âge
ages = []
with open('titanic.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['Age']:
            ages.append(float(row['Age']))

age_moyen = sum(ages) / len(ages)

# Insérer les données en remplaçant l'âge manquant par l'âge moyen
with open('titanic.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Ignorer l'en-tête
    for row in reader:
        row = [age_moyen if x == '' and i == 5 else x for i, x in enumerate(row)]  # i == 5 correspond à l'index de la colonne Age
        cur.execute(
            "INSERT INTO titanic (PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )

# Valider et fermer
conn.commit()
cur.close()
conn.close()

